$\textbf{PROGRAMMING #3 ASSIGNMENT}$
---
Instructions:
-

1. Read the article: https://www.sciencedirect.com/science/article/abs/pii/S0031320322001753
2. Replicate the study using the same dataset.
3. Read articles about Adjusted Rand Index, Normalized Mutual Information, and Folkes-Mallows Index (only use paper published in IEEE, sciencedirect, springerlink, Taylor Francis).
4. Aside from the Adjusted Rand Index (ARI), and Normalized Mutual Information (NMI), use the Folkes-Mallows Index (FMI), and compare the result of each performance index.
5. Compare and contrast each performance index, what are the advantages and disadvantages of ARI, NMI, and FMI, and when to use each?
6. Using Kmodes and Hierarchical Clustering, use the same dataset and perform categorical data clustering, use FMI, ARI, and NMI for the comparison of performance.
7. Write your report using Latex. Your report should be focused on the "why's and the what's" of each performance metrices (i.e. why is FMI always greater than ARI and NMI? What's the problem with ARI and NMI?). 

<br>
Name: Sheena Stella Salde <br>

# Section: Configuration and Initial Setup

Description: This section sets up necessary libraries, configurations and url of datasets

In [96]:
# Import necessary libraries
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import random
import requests
from itertools import combinations
from io import StringIO
from kmodes.kmodes import KModes
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF
from sklearn.manifold import SpectralEmbedding, TSNE
from sklearn.metrics import adjusted_rand_score as ARI, normalized_mutual_info_score as NMI, fowlkes_mallows_score as FMI
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.models import Sequential
from keras.layers import Input, Dense
from sklearn.cluster import AgglomerativeClustering
import warnings

In [97]:
# Ignore all warnings
warnings.filterwarnings('ignore')

In [12]:
# Dataset urls
#Cannot find the dataset for COIL20

datasets = {
    "Soybean": "https://archive.ics.uci.edu/static/public/91/data.csv", #Soybean (Small)
    "Zoo": "https://archive.ics.uci.edu/static/public/111/data.csv", #Zoo
    "Heart Disease": "https://archive.ics.uci.edu/static/public/45/data.csv", #Heart Disease
    "Breast Cancer": "https://archive.ics.uci.edu/static/public/15/data.csv", # Breast Cancer Wisconsin (Original)
    "Dermatology": "https://archive.ics.uci.edu/static/public/33/data.csv", #Dermatology
    "Letters(E, F)": "https://archive.ics.uci.edu/static/public/59/data.csv", #Letter Recognition (E, F)
    "DNA": "https://archive.ics.uci.edu/static/public/69/data.csv", #Molecular Biology (Splice-junction Gene Sequences)
    "Mushroom": "https://archive.ics.uci.edu/static/public/73/data.csv", #Mushroom
    "Iris": "https://archive.ics.uci.edu/static/public/53/data.csv", #Iris
    "Isolet": "https://archive.ics.uci.edu/static/public/54/data.csv", #ISOLET (5)
    "Optical": "https://archive.ics.uci.edu/static/public/80/data.csv", #Optical Recognition of Handwritten Digits
    "PenDigits": "https://archive.ics.uci.edu/static/public/81/data.csv" #Pen-Based Recognition of Handwritten Digits
}

In [3]:
# Set the true number of cluster for each dataset
num_clusters_dict = {
    "Soybean": 4,
    "Zoo": 7,
    "Heart Disease": 2,
    "Breast Cancer": 2,
    "Dermatology": 6,
    "Letters(E, F)": 2,
    "DNA": 3,
    "Mushroom": 2,
    "Iris": 3,
    "Isolet": 26,
    "Optical": 10,
    "PenDigits": 10
}

In [4]:
# List of datasets for clustering ensemble task
datasets_ensemble = [
    "Iris",
    "Isolet",
    "Optical",
    "PenDigits"
]

In [5]:
# Set the number of runs for benchmarking
num_runs = 10

In [6]:
k_means_runs = 60  # Number of k-means runs for clustering ensemble

In [7]:
df_dic = {}  # Dictionary to store cleaned dataframes

# Dataset Gathering

In [ ]:
# # Dictionary to hold raw dataframes
raw_dataframes = {}

In [13]:
raw_dataframes["Soybean"] = pd.read_csv("https://archive.ics.uci.edu/static/public/91/data.csv")

In [14]:
raw_dataframes["Zoo"] = pd.read_csv("https://archive.ics.uci.edu/static/public/111/data.csv")

In [15]:
raw_dataframes["Heart Disease"] = pd.read_csv("https://archive.ics.uci.edu/static/public/45/data.csv")

In [17]:
raw_dataframes["Breast Cancer"] = pd.read_csv("https://archive.ics.uci.edu/static/public/15/data.csv")

In [18]:
raw_dataframes["Dermatology"] = pd.read_csv("https://archive.ics.uci.edu/static/public/33/data.csv")

In [19]:
raw_dataframes["Letters(E, F)"] = pd.read_csv("https://archive.ics.uci.edu/static/public/59/data.csv")

In [102]:
raw_dataframes["DNA"] = pd.read_csv("https://archive.ics.uci.edu/static/public/69/data.csv")

In [21]:
raw_dataframes["Mushroom"] = pd.read_csv("https://archive.ics.uci.edu/static/public/73/data.csv")

In [26]:
raw_dataframes["Iris"] = pd.read_csv("https://archive.ics.uci.edu/static/public/53/data.csv")

In [27]:
raw_dataframes["Isolet"] = pd.read_csv("isolet5.data", nrows=1560)

In [28]:
raw_dataframes["Optical"] = pd.read_csv("https://archive.ics.uci.edu/static/public/80/data.csv")

In [29]:
raw_dataframes["PenDigits"] = pd.read_csv("https://archive.ics.uci.edu/static/public/81/data.csv")

In [30]:
# # Dictionary to hold raw dataframes
# raw_dataframes = {}

# # # Load each dataset into a dataframe and store it in the raw_dataframes dictionary
# for name, url in datasets.items():
#     df = pd.read_csv(url)
#     raw_dataframes[name] = df


In [31]:
raw_dataframes["Soybean"].head(1)

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,...,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots,class
0,4,0,2,1,1,1,0,1,0,2,...,0,0,4,0,0,0,0,0,0,D1


In [32]:
raw_dataframes["Zoo"].head(1)

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1


In [33]:
raw_dataframes["Heart Disease"].head(1)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0


In [34]:
raw_dataframes["Dermatology"].head(1)

,erythema,scaling,definite-borders,itching,koebner phenomenon,polygonal papules,follicular papules,oral-mucosal involvement,knee elbow involvement,scalp involvement,...,disappearance of the granular layer,vacuolisation and damage of the basal layer,spongiosis,saw-tooth appearance of retes,follicular horn plug,perifollicular parakeratosis,inflammatory monoluclear infiltrate,band-like infiltrate,age,class
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55.0,2


In [35]:
raw_dataframes["Letters(E, F)"].head(1)

,lettr,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8


In [103]:
raw_dataframes["DNA"].head(1)

,class,instancename,Base1,Base2,Base3,Base4,Base5,Base6,Base7,Base8,...,Base51,Base52,Base53,Base54,Base55,Base56,Base57,Base58,Base59,Base60
0,EI,ATRINS-DONOR-521,C,C,A,G,C,T,G,C,...,A,G,C,C,A,G,T,C,T,G


In [37]:
raw_dataframes["Mushroom"].head(1)

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,poisonous
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p


In [38]:
raw_dataframes["Iris"].head(1)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa


In [46]:
raw_dataframes["Isolet"].head()

,-0.2080,0.3480,0.3280,0.5040,0.9320,1.0000,0.8360,0.6680,0.2720,0.2400,...,0.2500,-0.0624,0.2188,0.4532,0.1094,0.1718,0.1562,0.0468,-0.3750,1.
0,-0.2864,0.1992,0.2822,0.4398,0.7012,0.7800,1.0000,0.9792,0.5850,0.4066,...,-0.0078,-0.1472,-0.1782,0.0078,0.1162,-0.0542,-0.0542,-0.0388,-0.7984,1.0
1,-0.2348,0.3826,0.6142,0.7492,0.0546,-0.4020,-0.3504,-0.2990,-0.6848,-0.6528,...,0.2834,0.1500,0.0834,-0.2000,-0.1834,0.0500,-0.0166,-0.1834,-0.8666,2.0
2,-0.1856,0.3592,0.7126,0.7366,0.3414,0.1018,-0.1556,-0.2514,-0.2514,-0.3892,...,0.2840,0.5556,0.4568,0.4568,0.4568,0.2098,0.0370,-0.0618,-0.3334,2.0
3,-0.1814,0.4404,0.8394,1.0000,0.7564,0.1866,0.0260,-0.0726,-0.2124,-0.3730,...,0.1688,-0.1688,0.2728,0.2988,0.2468,0.1948,-0.0130,-0.2988,-0.7662,3.0
4,-0.3212,0.6242,0.6424,0.6666,0.5090,0.1454,0.0060,-0.1454,-0.2606,-0.1940,...,0.4528,0.3584,0.4906,0.2830,0.3584,0.6792,0.3018,0.1698,-0.2642,3.0


In [40]:
raw_dataframes["Optical"].head(1)

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,...,Attribute56,Attribute57,Attribute58,Attribute59,Attribute60,Attribute61,Attribute62,Attribute63,Attribute64,class
0,0,1,6,15,12,1,0,0,0,7,...,0,0,0,6,14,7,1,0,0,0


In [43]:
raw_dataframes["PenDigits"].head()

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Class
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


# Data Preprocessing

In [44]:
# Dictionary to hold processed data
processed_dataframes = {}

# Process each dataframe in the raw_dataframes dictionary
for name, df in raw_dataframes.items():
    if name == "Letters(E, F)":
        y = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    elif name == "Mushroom":
        y = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    elif name == "DNA":
        y = df.iloc[:, 0].str.strip()
        # Remove the second column from the DNA dataframe
        X = df.iloc[:, 1:].drop(df.columns[1], axis=1)
    else:
        X, y = df.iloc[:, :-1], df.iloc[:, -1]

    # Limit rows for "Letters(E, F)" dataset
    if name == "Letters(E, F)" and len(X) > 1543:
        X = X.iloc[:1543]
        y = y.iloc[:1543]


    if name == "Isolet" and len(X) > 1560:
        X = X.iloc[:1560]
        y = y.iloc[:1560]
        
    # Drop columns with only 1 unique value
    for col in X.columns:
        if len(X[col].unique()) <= 1:
            X.drop(columns=[col], inplace=True)

    print(f"Before dropping rows with NaNs in {name}:")
    print(f"Shape of features (X): {X.shape}")
    print(f"Shape of targets (y): {y.shape}")
    print("NaN counts per column:\n", X.isnull().sum())

    # Drop rows that contain any NaN values
    X.dropna(axis=0, how='any', inplace=True)
    # After dropping NaNs, ensure y is aligned with X
    y = y[X.index]

    print(f"After dropping rows with NaNs in {name}:")
    print(f"Shape of features (X): {X.shape}")
    print(f"Shape of targets (y): {y.shape}")
    print("NaN counts per column:\n", X.isnull().sum())


    # Store processed data in processed_dataframes dictionary
    processed_dataframes[name] = {'features': X, 'targets': y}

Before dropping rows with NaNs in Soybean:
Shape of features (X): (47, 21)
Shape of targets (y): (47,)
NaN counts per column:
 date               0
plant-stand        0
precip             0
temp               0
hail               0
crop-hist          0
area-damaged       0
severity           0
seed-tmt           0
germination        0
leaves             0
lodging            0
stem-cankers       0
canker-lesion      0
fruiting-bodies    0
external-decay     0
mycelium           0
int-discolor       0
sclerotia          0
fruit-pods         0
roots              0
dtype: int64
After dropping rows with NaNs in Soybean:
Shape of features (X): (47, 21)
Shape of targets (y): (47,)
NaN counts per column:
 date               0
plant-stand        0
precip             0
temp               0
hail               0
crop-hist          0
area-damaged       0
severity           0
seed-tmt           0
germination        0
leaves             0
lodging            0
stem-cankers       0
canker-lesion      0

/var/folders/gz/h6s5flr16sqcrdbrjz0xxdc40000gn/T/ipykernel_49341/518452889.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=[col], inplace=True)
/var/folders/gz/h6s5flr16sqcrdbrjz0xxdc40000gn/T/ipykernel_49341/518452889.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(axis=0, how='any', inplace=True)
/var/folders/gz/h6s5flr16sqcrdbrjz0xxdc40000gn/T/ipykernel_49341/518452889.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [47]:
def preprocess_datasets(dataframes):
    if 'Zoo' in dataframes:
        zoo_df = dataframes['Zoo']['features']
        # Assuming columns are indexed by integers and the first column is indeed 0
        # Check if the first column exists, otherwise drop using the actual column name
        if 0 in zoo_df.columns:
            zoo_df = zoo_df.drop(columns=[0])
        else:
            zoo_df = zoo_df.drop(columns=zoo_df.columns[0])
        dataframes['Zoo']['features'] = zoo_df

    if 'Heart Disease' in dataframes:
        hd_df = dataframes['Heart Disease']['features']
        # Safely check if these columns exist before attempting to drop them
        columns_to_drop = [hd_df.columns[i] for i in [0, 3, 4, 7, 9] if i < len(hd_df.columns)]
        hd_df = hd_df.drop(columns=columns_to_drop)
        dataframes['Heart Disease']['features'] = hd_df
        y_hd = dataframes['Heart Disease']['targets']
        dataframes['Heart Disease']['targets'] = y_hd.apply(lambda x: 0 if x == 0 else 1)
    
    if 'Breast Cancer' in dataframes:
        bcw_df = dataframes['Breast Cancer']['features']
        bcw_df = bcw_df.drop(columns=bcw_df.columns[0])
        dataframes['Breast Cancer']['features'] = bcw_df
    
    if 'Dermatology' in dataframes:
        derm_df = dataframes['Dermatology']['features']
        derm_df = derm_df.drop(columns=derm_df.columns[-1])
        dataframes['Dermatology']['features'] = derm_df

    if 'Letters(E, F)' in dataframes:
        lr_ef_df = dataframes['Letters(E, F)']['features']
        lr_ef_targets = dataframes['Letters(E, F)']['targets']
        mask = lr_ef_targets.isin(['E', 'F'])
        dataframes['Letters(E, F)']['features'] = lr_ef_df[mask]
        dataframes['Letters(E, F)']['targets'] = lr_ef_targets[mask]

    return dataframes


In [48]:
# Apply preprocessing to your datasets
dataframes = preprocess_datasets(processed_dataframes)

In [49]:
processed_dataframes["DNA"]

{'features':      Base1 Base2 Base3 Base4 Base5 Base6 Base7 Base8 Base9 Base10  ... Base51  \
 0        C     C     A     G     C     T     G     C     A      T  ...      A   
 1        A     G     A     C     C     C     G     C     C      G  ...      G   
 2        G     A     G     G     T     G     A     A     G      G  ...      C   
 3        G     G     G     C     T     G     C     G     T      T  ...      G   
 4        G     C     T     C     A     G     C     C     C      C  ...      C   
 ...    ...   ...   ...   ...   ...   ...   ...   ...   ...    ...  ...    ...   
 3185     T     C     T     C     T     T     C     C     C      T  ...      T   
 3186     G     A     G     C     T     C     C     C     A      G  ...      G   
 3187     T     C     T     C     G     G     G     G     G      C  ...      C   
 3188     A     T     T     C     T     A     C     T     T      A  ...      A   
 3189     A     G     G     C     T     G     C     C     T      A  ...      A   
 
  

# Clustering Ensemble on Numerical Datasets

In [50]:
def run_multiple_kmeans(features, n_clusters, n_runs):
    all_labels = []
    for i in range(n_runs):
        random_state = random.randint(0, 1000)
        kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=random_state, n_init=10)
        labels = kmeans.fit_predict(features)
        all_labels.append(labels)
    return np.array(all_labels).T

In [51]:
for dataset_name in datasets_ensemble:
    # Extracting features and targets from the preloaded datasets
    features = dataframes[dataset_name]["features"]
    targets = dataframes[dataset_name]["targets"]

    # Converting targets to numerical labels if they aren't already
    if targets.dtype.kind in 'O':  # Check if targets are object type (e.g., strings)
        targets = LabelEncoder().fit_transform(targets)

    # Determine the number of clusters from the unique elements in targets
    n_clusters = len(np.unique(targets))

    # Run multiple k-means and collect results
    ensembled_features = run_multiple_kmeans(features, n_clusters, num_runs)
    
    # Convert numpy array to DataFrame and replace the original data
    ensembled_features_df = pd.DataFrame(ensembled_features, index=features.index)
    dataframes[dataset_name]["features"] = ensembled_features_df

# Clustering Algorithms and Helper Functions



In [107]:
def perform_agglomerative_clustering(data, n_clusters):
    """Perform clustering using Agglomerative clustering on potentially mixed data types using Label Encoding."""
    # Identifying categorical columns
    categorical_cols = data.select_dtypes(include=[object]).columns.tolist()

    # Label encoding for categorical variables
    if categorical_cols:
        for col in categorical_cols:
            le = LabelEncoder()
            data[col] = le.fit_transform(data[col])

    # Extract features as an array
    features = data.values

    # Create the model with the specified number of clusters and using the 'ward' linkage method
    model = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')

    # Fitting the model to the data and getting cluster labels
    clusters = model.fit_predict(features)
    
    return clusters

In [53]:
def perform_kmodes(features, n_clusters):
    """Perform clustering using KModes algorithm."""
    km = KModes(n_clusters=n_clusters, init='random', n_init=5)
    clusters = km.fit_predict(features)
    return clusters

In [54]:
def perform_ordinal_encoding(features, true_labels, n_clusters):
    """Perform Ordinal Encoding followed by clustering."""
    encoder = LabelEncoder()
    features_encoded = features.apply(encoder.fit_transform)
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    return kmeans.fit_predict(features_encoded, n_clusters)

In [67]:
def perform_one_hot_encoding(features, true_labels, n_clusters):
    """Perform One-Hot Encoding followed by clustering."""
    encoder = OneHotEncoder()
    features_encoded = encoder.fit_transform(features).toarray()
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    return kmeans.fit_predict(features_encoded, n_clusters)

In [68]:
def perform_link(features, n_clusters):
    encoder = OneHotEncoder()
    features_encoded = encoder.fit_transform(features).toarray()
    # Calculate pairwise dissimilarities (1 - similarity), ensuring non-negative distances
    ochiai_distance = pdist(features_encoded, lambda u, v: max(0, 1 - ochiai_coefficient_for_link(u, v)))
    link_matrix = linkage(ochiai_distance, method='average')
    clusters = fcluster(link_matrix, t=n_clusters, criterion='maxclust')
    return clusters

In [69]:
def ochiai_coefficient_for_link(b1, b2):
    intersection = np.dot(b1, b2)
    norm_b1 = np.sqrt(np.dot(b1, b1))
    norm_b2 = np.sqrt(np.dot(b2, b2))
    denominator = (norm_b1 * norm_b2)
    if denominator == 0:
        return 0  # Return 0 if either or both vectors are all zeros
    return intersection / denominator

In [70]:
def perform_cde(features, n_clusters):
    """Perform Categorical Data Embedding and clustering using t-SNE and k-Means."""
    encoder = OneHotEncoder(sparse_output=False)
    features_encoded = encoder.fit_transform(features)
    tsne_model = TSNE(n_components=2, perplexity=30, learning_rate=200)
    features_embedded = tsne_model.fit_transform(features_encoded)
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    clusters = kmeans.fit_predict(features_embedded)
    return clusters

In [71]:
def perform_hierarchical_clustering(features, n_clusters, method='ward'):
    """Perform Hierarchical Clustering with handling for '?' characters and categorical variables."""
    
    # Handle '?' characters and replace them appropriately
    for column in features.columns:
        if features[column].dtype == object:
            # Check if the column contains '?'
            if '?' in features[column].unique():
                if features[column].str.isnumeric().any():
                    # Assume numeric column with missing values represented as '?'
                    # Convert '?' to NaN and then fill with the mean of the column
                    features[column] = pd.to_numeric(features[column], errors='coerce')
                    features[column].fillna(features[column].mean(), inplace=True)
                else:
                    # Categorical column
                    mode_value = features[column].mode()[0]
                    features[column] = features[column].replace('?', mode_value)
            # Encode categorical variables
            le = LabelEncoder()
            features[column] = le.fit_transform(features[column].astype(str))
        else:
            # Directly fill missing values with mean if any
            if features[column].isnull().any():
                features[column].fillna(features[column].mean(), inplace=True)

    # Ensure all data is in float format to avoid conversion errors in linkage
    features = features.astype(float)

    # Create the linkage matrix
    Z = linkage(features, method=method)

    # Create clusters by cutting the dendrogram at the specified number of clusters
    clusters = fcluster(Z, t=n_clusters, criterion='maxclust')
    return clusters

In [72]:
def perform_cdc_dr(features, n_clusters, embedding_method='SE', operation='Joint'):
    """
    Perform CDC_DR algorithm with specified graph embedding method and operation.
    :param features: DataFrame of features
    :param n_clusters: Number of clusters to form
    :param embedding_method: 'NE', 'SE', 'NMF', or 'AE'
    :param operation: 'Joint' or 'Mean'
    :return: clusters - Cluster labels for each sample
    """
    
    # Construct similarity graph from features
    graph = construct_similarity_graph(features)

    # Apply graph embedding technique
    embedded_graph = graph_embedding(graph, method=embedding_method)

    value_to_index = create_value_to_index_mapping(features)
    
    # Ensure integrated_data is 2D before clustering
    if operation == 'Joint':
        integrated_data = joint_operation(embedded_graph, features, value_to_index)
    elif operation == 'Mean':
        integrated_data = mean_operation(embedded_graph, features, value_to_index)
    else:
        raise ValueError("Operation must be either 'Joint' or 'Mean'.")

    # Cluster the integrated data
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    clusters = kmeans.fit_predict(integrated_data)

    return clusters

In [73]:
def construct_similarity_graph(features):
    """
    Construct a similarity graph from features based on categorical values.
    :param features: DataFrame of features, each row is a sample and columns are categorical features
    :return: graph - A NetworkX graph with nodes representing categorical values and weighted edges
    """
    # Step 1: Prepare all unique categorical values and their indices
    unique_values_dict = {}
    for column in features:
        unique_values = np.unique(features[column])
        for val in unique_values:
            unique_values_dict[f"{column}_{val}"] = np.where(features[column] == val)[0]
    
    # Step 2: Calculate similarity between all pairs of unique categorical values
    graph = nx.Graph()
    for (val1, indices1), (val2, indices2) in combinations(unique_values_dict.items(), 2):
        # Calculate similarity (e.g., using Ochiai coefficient)
        sim = ochiai_coefficient(indices1, indices2)  # Define this function based on your chosen similarity metric
        if sim > 0:  # If the similarity is non-zero, add an edge
            graph.add_edge(val1, val2, weight=sim)
    
    # Add all nodes explicitly in case some have no edges
    for val in unique_values_dict.keys():
        if val not in graph:
            graph.add_node(val)

    return graph

In [74]:
def ochiai_coefficient(indices1, indices2):
    """
    Calculate Ochiai coefficient between two sets of indices
    :param indices1: array-like list of indices for the first categorical value
    :param indices2: array-like list of indices for the second categorical value
    :return: Ochiai coefficient as float
    """
    set1 = set(indices1)
    set2 = set(indices2)
    intersection = len(set1.intersection(set2))
    if intersection == 0: return 0  # No overlap
    return intersection / np.sqrt(len(set1) * len(set2))  # Ochiai coefficient formula

In [75]:
def graph_embedding(graph, method='SE', dimensions=2):
    """
    Apply graph embedding method to the constructed graph.
    :param graph: NetworkX graph
    :param method: string representing the graph embedding method: 'NE', 'SE', 'NMF', 'AE'
    :param dimensions: the number of dimensions for the embedding
    :return: embedded_graph - An array-like embedded representation of the graph
    """
    # Convert graph to adjacency matrix and then to numpy ndarray
    adjacency_matrix = nx.to_numpy_array(graph)
    adjacency_matrix = np.asarray(adjacency_matrix)
    
    if method == 'NE':
        # Directly use the adjacency matrix as features (no embedding)
        embedded_graph = adjacency_matrix

    elif method == 'SE':
        # Apply Spectral Embedding
        embedding_model = SpectralEmbedding(n_components=dimensions)
        embedded_graph = embedding_model.fit_transform(adjacency_matrix)

    elif method == 'NMF':
        # Apply Non-negative Matrix Factorization for embedding
        model = NMF(n_components=dimensions, init='random', max_iter=10000)
        embedded_graph = model.fit_transform(adjacency_matrix)

    elif method == 'AE':
        # Apply Autoencoder for graph embedding
        n_nodes = adjacency_matrix.shape[0]
        # Define the autoencoder structure with an explicit Input layer
        autoencoder = Sequential([
            Input(shape=(n_nodes,)),
            Dense(64, activation='relu'),
            Dense(dimensions, activation='relu'),  # Embedding layer
            Dense(64, activation='relu'),
            Dense(n_nodes, activation='sigmoid')
        ])
        autoencoder.compile(optimizer='adam', loss='mse')
        adjacency_matrix_norm = adjacency_matrix / np.max(adjacency_matrix)  # Normalize adjacency matrix
        autoencoder.fit(adjacency_matrix_norm, adjacency_matrix_norm, epochs=50, verbose=0)
        # Define the encoder model by selecting the first two layers from the autoencoder
        encoder = Sequential(autoencoder.layers[:2])  
        embedded_graph = encoder.predict(adjacency_matrix_norm, verbose=0)

    else:
        raise NotImplementedError(f"Graph embedding method {method} is not implemented.")
    
    return np.array(embedded_graph)

In [76]:
def create_value_to_index_mapping(features):
    """
    Create a mapping from each unique categorical value to a unique index.
    :param features: DataFrame of features, each column is a categorical feature
    :return: Dictionary of value to index mapping
    """
    # Extracting unique values from each feature
    unique_values = set()
    for column in features.columns:
        unique_values.update(features[column].unique())

    # Creating a mapping from unique values to an index
    value_to_index = {value: idx for idx, value in enumerate(unique_values)}
    return value_to_index

In [77]:
def joint_operation(embedded_graph, features, value_to_index):
    # Concatenates the embeddings for each categorical value in each sample
    joint_embedded = []
    for _, row in features.iterrows():
        joint_vector = []
        for value in row:
            index = value_to_index[value]  # Map each categorical value to its index in the embedded graph
            joint_vector.extend(embedded_graph[index])
        joint_embedded.append(joint_vector)
    return np.array(joint_embedded)

In [78]:
def mean_operation(embedded_graph, features, value_to_index):
    # Calculates the mean of the embeddings for each categorical value in each sample
    mean_embedded = []
    for _, row in features.iterrows():
        vectors = [embedded_graph[value_to_index[value]] for value in row]
        mean_vector = np.mean(vectors, axis=0)
        mean_embedded.append(mean_vector)
    return np.array(mean_embedded)

# Runnning Clustering Algorithms K-modes
This section executes the defined clustering algorithms on the prepared datasets and collects the results.

In [91]:
def run_clustering_algorithms(dataframes, n_clusters_dict, num_runs=10):
    results_list = []
    for name, data in dataframes.items():
        print("Processing:", name)
        features = data['features']
        true_labels = data['targets'].squeeze()  # Assuming targets are in a single column
        n_clusters = n_clusters_dict.get(name, 2)  # Default to 2 clusters if not specified

        metrics = {'KModes': [], 'Ordinal': [], 'One-Hot': [], 'Link': [], 'CDE': [], 'Hierarchical': []}  # Initialize a dictionary to store results for each method

        # Include CDC_DR methods in metrics dictionary
        embedding_methods = ['NE', 'SE', 'NMF', 'AE']  # Non-Embedding, Spectral Embedding, Nonnegative Matrix Factorization, Autoencoder
        operations = ['Joint', 'Mean']  # The two types of operations
        
        for em in embedding_methods:
            for op in operations:
                key_name = f"CDC_DR+{em} ({op})"
                metrics[key_name] = []

        for _ in range(num_runs):
            # KModes
            km_clusters = perform_kmodes(features, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, km_clusters)
            metrics['KModes'].append((ari, nmi, fmi))

            # Ordinal Encoding
            ord_clusters = perform_ordinal_encoding(features, true_labels, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, ord_clusters)
            metrics['Ordinal'].append((ari, nmi, fmi))

            # One-Hot Encoding
            oh_clusters = perform_one_hot_encoding(features, true_labels, n_clusters)
            ari, nmi, fmi  = calculate_metrics(true_labels, oh_clusters)
            metrics['One-Hot'].append((ari, nmi, fmi))

            # Link with Ochiai Coefficient
            link_clusters = perform_link(features, n_clusters)
            ari, nmi, fmi  = calculate_metrics(true_labels, link_clusters)
            metrics['Link'].append((ari, nmi, fmi))

            # CDE with t-SNE and k-Means
            cde_clusters = perform_cde(features, n_clusters)
            ari, nmi, fmi  = calculate_metrics(true_labels, cde_clusters)
            metrics['CDE'].append((ari, nmi, fmi))

            # Hierarchical Clustering
            hier_clusters = perform_agglomerative_clustering(features, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, hier_clusters)
            metrics['Hierarchical'].append((ari, nmi, fmi))

            # CDC_DR with various embedding methods and operations
            for embedding_method in ['NE', 'SE', 'NMF', 'AE']:
                for operation in ['Joint', 'Mean']:
                    cdc_dr_clusters = perform_cdc_dr(features, n_clusters, embedding_method, operation)
                    ari, nmi, fmi  = calculate_metrics(true_labels, cdc_dr_clusters)
                    metrics[f"CDC_DR+{embedding_method} ({operation})"].append((ari, nmi, fmi))

        # Calculate mean and standard deviation for each method and append to results list
        for method, values in metrics.items():
            ari_vals, nmi_vals, fmi_vals = zip(*values)  # Unpack the metrics values
            ari_mean, ari_std = np.mean(ari_vals), np.std(ari_vals)
            nmi_mean, nmi_std = np.mean(nmi_vals), np.std(nmi_vals)
            fmi_mean, fmi_std = np.mean(fmi_vals), np.std(fmi_vals)  # Calculate mean and std for FMI
            results_list.append({
                "Dataset": name,
                "Method": method,
                "ARI": f"{ari_mean:.4f}±{ari_std:.2f}",
                "NMI": f"{nmi_mean:.4f}±{nmi_std:.2f}",
                "FMI": f"{fmi_mean:.4f}±{fmi_std:.2f}"  # Include FMI in results
            })
    # Convert list of dictionaries to DataFrame for results
    results_df = pd.DataFrame(results_list)
    return results_df

In [92]:
def calculate_metrics(true_labels, predicted_labels):
    """
    Calculate clustering metrics: Adjusted Rand Index, Normalized Mutual Information, and Fowlkes-Mallows Index.
    
    Args:
    true_labels (array-like): True cluster labels.
    predicted_labels (array-like): Cluster labels predicted by a clustering algorithm.
    
    Returns:
    tuple: A tuple containing the ARI, NMI, and FMI scores.
    """
    ari_score = ARI(true_labels, predicted_labels)
    nmi_score = NMI(true_labels, predicted_labels)
    fmi_score = FMI(true_labels, predicted_labels)
    return ari_score, nmi_score, fmi_score

In [98]:
def reformat_results(results_df):
    # Include 'FMI' in the list of columns to be melted along with 'ARI' and 'NMI'
    expanded_df = pd.melt(results_df, id_vars=["Dataset", "Method"], value_vars=["ARI", "NMI", "FMI"], var_name="Metric", value_name="Value")
    
    # Split the 'Value' column into 'Metric_Value' and 'Std', assuming the format "value±std"
    expanded_df[['Metric_Value', 'Std']] = expanded_df['Value'].str.split('±', expand=True)
    expanded_df.drop(columns=['Value'], inplace=True)  # Removing the original combined column
    
    # Convert the 'Metric_Value' and 'Std' columns to numeric types for further operations
    expanded_df['Metric_Value'] = expanded_df['Metric_Value'].astype(float)
    expanded_df['Std'] = expanded_df['Std'].astype(float)

    # Reformat 'Metric_Value' to combine mean and standard deviation into a single string formatted as needed
    expanded_df['Metric_Value'] = expanded_df['Metric_Value'].map('{:.4f}'.format) + "±" + expanded_df['Std'].map('{:.2f}'.format)
    
    # Ensuring the order of datasets and methods remains consistent with the original DataFrame
    dataset_order = results_df['Dataset'].unique()
    method_order = results_df['Method'].unique()

    # Creating a pivot table to restructure the DataFrame
    # This pivot table organizes the data by 'Dataset' and 'Metric', with methods as columns and metric values as cell data
    pivot_df = expanded_df.pivot_table(index=["Dataset", "Metric"], columns="Method", values="Metric_Value", aggfunc='first')
    
    # Reindexing the pivot table to maintain the original order
    pivot_df = pivot_df.reindex(dataset_order, level='Dataset')
    pivot_df = pivot_df.reindex(method_order, axis='columns')

    return pivot_df

In [108]:
# Running all algorithms and storing the results
results = run_clustering_algorithms(dataframes, num_clusters_dict, num_runs)

Processing: Soybean
Processing: Zoo
Processing: Heart Disease
Processing: Breast Cancer
Processing: Dermatology
Processing: Letters(E, F)
Processing: DNA
Processing: Mushroom
Processing: Iris
Processing: Isolet
Processing: Optical
Processing: PenDigits


In [109]:
# Use the function to reformat the results
formatted_results = reformat_results(results)

# Presentation of Results

Number of features for numerical datasets are based on the # of runs of kmeans in the clustering ensemble. For an accurate measure of the original versions of the datasets, run this before running the clustering ensemble.

In [113]:
# Prepare data for the table
table_data = []

for name, content in dataframes.items():
    X, y = content['features'], content['targets']
    table_data.append({
        "Name": name,
        "Number of Samples": X.shape[0],
        "Number of Features": X.shape[1],
        "Number of Unique Values in Target": len(pd.unique(y))
    })

# Convert table data into a DataFrame for pretty printing
table_df = pd.DataFrame(table_data)
table_df

,Name,Number of Samples,Number of Features,Number of Unique Values in Target
0,Soybean,47,21,4
1,Zoo,101,16,7
2,Heart Disease,297,8,2
3,Breast Cancer,683,9,2
4,Dermatology,358,33,6
5,"Letters(E, F)",103,16,2
6,DNA,3190,60,3
7,Mushroom,5644,21,6
8,Iris,150,10,3
9,Isolet,1558,10,26


In [112]:
# Print the formatted results
formatted_results


Method                      KModes      Ordinal       One-Hot          Link  \
Dataset       Metric                                                          
Soybean       ARI      0.9571±0.07  0.5452±0.00   1.0000±0.00   1.0000±0.00   
              FMI      0.9677±0.05  0.6568±0.00   1.0000±0.00   1.0000±0.00   
              NMI      0.9703±0.05  0.7158±0.00   1.0000±0.00   1.0000±0.00   
Zoo           ARI      0.6307±0.09  0.6915±0.12   0.7434±0.10   0.8893±0.00   
              FMI      0.7114±0.07  0.7597±0.10   0.8022±0.08   0.9162±0.00   
              NMI      0.7771±0.04  0.7898±0.05   0.8484±0.04   0.8731±0.00   
Heart Disease ARI      0.3637±0.02  0.4471±0.00   0.4310±0.01   0.0013±0.00   
              FMI      0.6859±0.01  0.7251±0.00   0.7156±0.00   0.6966±0.00   
              NMI      0.2834±0.01  0.3519±0.00   0.3404±0.01   0.0017±0.00   
Breast Cancer ARI      0.7328±0.05  0.8503±0.00   0.8098±0.00   0.0101±0.00   
              FMI      0.8835±0.02  0.9324±0.00   0.9113±0.00   0.7367±0.00   
              NMI      0.6311±0.04  0.7525±0.00   0.7363±0.00   0.0181±0.00   
Dermatology   ARI      0.5094±0.04  0.7096±0.00   0.8912±0.09   0.5902±0.00   
              FMI      0.6073±0.03  0.7675±0.00   0.9129±0.08   0.7274±0.00   
              NMI      0.6168±0.04  0.8754±0.00   0.9181±0.03   0.7449±0.00   
Letters(E, F) ARI      0.4044±0.06  0.6748±0.01   0.3062±0.37   0.2201±0.00   
              FMI      0.7023±0.03  0.8396±0.00   0.6638±0.18   0.6736±0.00   
              NMI      0.3303±0.05  0.6023±0.01   0.2932±0.35   0.3115±0.00   
DNA           ARI      0.0233±0.01  0.0362±0.02   0.5924±0.03   0.0422±0.00   
              FMI      0.3760±0.01  0.4092±0.00   0.7425±0.02   0.4874±0.00   
              NMI      0.0371±0.02  0.0524±0.01   0.5444±0.03   0.1253±0.00   
Mushroom      ARI     -0.0141±0.00  0.0005±0.00  -0.0175±0.00  -0.0175±0.00   
              FMI      0.5230±0.00  0.4724±0.00   0.5229±0.00   0.5229±0.00   
              NMI      0.0204±0.00  0.0115±0.00   0.0239±0.00   0.0239±0.00   
Iris          ARI      0.7302±0.00  0.7302±0.00   0.7302±0.00   0.7302±0.00   
              FMI      0.8208±0.00  0.8208±0.00   0.8208±0.00   0.8208±0.00   
              NMI      0.7582±0.00  0.7582±0.00   0.7582±0.00   0.7582±0.00   
Isolet        ARI      0.4294±0.01  0.3885±0.01   0.4644±0.00   0.4616±0.00   
              FMI      0.4557±0.01  0.4134±0.01   0.4859±0.00   0.4877±0.00   
              NMI      0.6964±0.01  0.6443±0.00   0.7160±0.00   0.7171±0.00   
Optical       ARI      0.6743±0.01  0.6692±0.00   0.6704±0.00   0.6704±0.00   
              FMI      0.7095±0.01  0.7053±0.00   0.7063±0.00   0.7063±0.00   
              NMI      0.7578±0.00  0.7547±0.00   0.7566±0.00   0.7566±0.00   
PenDigits     ARI      0.5572±0.03  0.6052±0.00   0.5320±0.00   0.5320±0.00   
              FMI      0.6080±0.03  0.6465±0.00   0.5874±0.00   0.5874±0.00   
              NMI      0.6867±0.01  0.7079±0.00   0.6821±0.00   0.6821±0.00   

Method                        CDE  Hierarchical CDC_DR+NE (Joint)  \
Dataset       Metric                                                
Soybean       ARI     0.9622±0.05   1.0000±0.00       1.0000±0.00   
              FMI     0.9715±0.04   1.0000±0.00       1.0000±0.00   
              NMI     0.9708±0.04   1.0000±0.00       1.0000±0.00   
Zoo           ARI     0.5636±0.02   0.6958±0.00       0.7545±0.08   
              FMI     0.6595±0.01   0.7641±0.00       0.8106±0.06   
              NMI     0.7655±0.01   0.7935±0.00       0.8366±0.03   
Heart Disease ARI     0.3406±0.02   0.2770±0.00       0.4203±0.00   
              FMI     0.6709±0.01   0.6403±0.00       0.7143±0.00   
              NMI     0.2641±0.02   0.2097±0.00       0.3323±0.00   
Breast Cancer ARI     0.8092±0.07   0.8690±0.00       0.8360±0.00   
              FMI     0.9112±0.03   0.9393±0.00       0.9248±0.00   
              NMI     0.7236±0.07   0.8031±0.00       0.7347±0.00   
Dermatology   ARI     0.7268±0.

# Questions

The Adjusted Rand Index (ARI), Normalized Mutual Information (NMI), and Fowlkes-Mallows Index (FMI) are all metrics used to evaluate the performance of clustering algorithms by comparing the clustering results with ground truth labels.

## Analysis
This analysis explores how well clustering algorithms perform by comparing their results to true labels using three methods: Adjusted Rand Index (ARI), Normalized Mutual Information (NMI), and Fowlkes-Mallows Index (FMI). The report found that the choice of operation (like averaging) and data size can affect the results. Specifically, ARI and NMI might be lower for small datasets with averaging, but they improve with more data. This suggests that the number of data points might influence these metrics, but further investigation is needed. It's interesting to note that FMI, which wasn't originally included, showed good results similar to ARI.

## Use of FMI, ARI AND NMI 
Descriptions
#### FMI (Fowlkes-Mallows Index): 
This metric looks at the balance between correctly assigned data points (true positives and negatives) compared to the true clusters. It focuses on the proportion of correct predictions and is less sensitive to false positives/negatives compared to ARI and NMI.
#### ARI (Adjusted Rand Index): 
This metric compares predicted and true clusters by considering pairs of data points. It checks how often pairs are assigned together (or separately) in both predicted and true clusters. ARI is stricter than other metrics because it adjusts for random chance, making it lower when clusters aren't perfectly aligned with true labels.
#### NMI (Normalized Mutual Information):
This metric measures the amount of information shared between clusters, considering cluster sizes. It is normalized to account for the potential maximum information, which can affect raw scores. NMI can be higher for some clustering methods because it evaluates how well clusters capture the data's inherent structure, even if they aren't perfectly separated. It's better suited for complex or overlapping cluster structures compared to ARI or FMI.

### When to Use:

#### FMI is preferred in some cases: 
Because FMI focuses on the proportion of correct assignments and doesn't heavily penalize false positives/negatives, it can have higher values compared to ARI and NMI, especially when there are many true negative pairings in the data.
##### ARI is lower due to its adjustment: 
ARI considers random chance, making it a more critical metric. This can lead to lower scores when clusters aren't perfectly aligned with true labels.
#### NMI can be higher or lower: 
NMI depends on the actual information shared between clusters. It can be higher for methods that capture the data's structure well, even with imperfect separation. It can handle complex cluster structures better than other metrics.